# For Handling Wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\sushant\\Jupyter Notebook\\Intern_Project\\LnagChain_chatbot\\RAG Chatbot\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [5]:
wiki.name

'wikipedia'

In [6]:
### web based loader

from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
## load,chunk and index the content of the html page

loader = WebBaseLoader("https://docs.smith.langchain.com/")

docs =loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
documents


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick 

In [7]:
# from sentence_transformers import SentenceTransformer
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
# model = "sentence-transformers/all-mpnet-base-v2"

# hf = HuggingFaceEmbeddings(
#     model_name = model,
#     model_kwargs = {'device' : 'cpu'},
#     encode_kwargs = {'normalize_embeddings' : False},
# )

In [8]:
# # making the index of faiss
# import faiss
# index = faiss.IndexFlatL2(len(hf.embed_query("Hello World")))

In [9]:
# from langchain_community.docstore.in_memory import InMemoryDocstore

# # vector store combinding the index of faiss and embedding form our model
# vector_store = FAISS(
#     embedding_function = hf,
#     index = index,
#     docstore = InMemoryDocstore(),
#     index_to_docstore_id = {},
# )

# ## FAISS Vector Database - using OPENAI embeddings
# # just making and storing the first fifteen chunks of the data 

# vector_store.add_documents(documents=documents)

In [10]:
# from langchain.embeddings import OpenAIEmbeddings  # or your specific embedding model

# # Initialize the embedding model
# embeddings = OpenAIEmbeddings()  # Replace with the actual embedding class you used earlier

# # Save the vector store
# vector_store.save_local("FAISS_STORE1")

# # Load the vector store with embeddings and allow dangerous deserialization
# loaded_vector_store = vector_store.load_local(
#     "FAISS_STORE1", embeddings=embeddings, allow_dangerous_deserialization=True
# )


In [11]:
# retriver = vector_store.as_retriever()
# retriver

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriver=vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000170D89F88B0>)

# Create Retriver tool

In [13]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(retriver, "langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [14]:
print(retriver_tool, retriver_tool.name)

name='langsmith_search' description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!' args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'> func=functools.partial(<function _get_relevant_documents at 0x00000170D15390D0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000170D89F88B0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n') coroutine=functools.partial(<function _aget_relevant_documents at 0x00000170D1539310>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000170D89F88B0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n') langsmith_search


### arvix -> website where all the research papers have been uploaded

In [15]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

## Combine langchain tools

In [16]:
tools = [wiki, arxiv, retriver_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\sushant\\Jupyter Notebook\\Intern_Project\\LnagChain_chatbot\\RAG Chatbot\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000170D15390D0>, retriever=VectorStoreRetriever(tags=['FAISS', 'Op

# query from this tool with the help of agents

In [18]:
from langchain.llms import Ollama
from langchain_openai import ChatOpenAI

# Initialize the LLM model (replace with the appropriate model name if different)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Test the model with a sample prompt
try:
    response = llm("Hello, world!")  # Input your test prompt here
    print("Response:", response)
except Exception as e:
    print("Error:", e)


C:\Users\HP\AppData\Local\Temp\ipykernel_9696\2142622769.py:9: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm("Hello, world!")  # Input your test prompt here


Response: content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_7f6be3efb0', 'finish_reason': 'stop', 'logprobs': None} id='run-eafb972c-aedd-4f8a-8d2c-5289542f83fc-0' usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20}


## PROMPT

In [19]:
from langchain import hub

# Get the prompt to use - we can modify this!!

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

## Agent Executer

In [21]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent, tools=tools, verbose = True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_comm

In [22]:
print(f"Agent: {agent}")
print(f"Tools: {tools}")


Agent: first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
}) middle=[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f

In [23]:
agent_executer.invoke({"input":"Tell me about Langsmith"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<you

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform designed for building production-grade LLM (Large Language Model) applications. It provides tools for closely monitoring and evaluating your application, enabling you to deploy quickly and confidently. LangSmith operates independently and does not require the use of LangChain's open-source frameworks, although it can integrate with them.\n\nKey features of LangSmith include:\n\n1. **Installation**: LangSmith can be installed using Python or TypeScript with package managers like pip or yarn.\n\n2. **API Key Creation**: Users need to create an API key through the platform's settings to access its features.\n\n3. **Environment Setup**: Users can set up their environment by exporting necessary API keys and configurations.\n\n4. **Logging and Evaluation**: LangSmith allows users to log traces and evaluate their applications through its experiments UI. It provides tutorials, conceptual guides, and how-to guides for bett

In [24]:
agent_executer.invoke({"input":"Tell me about Machine Learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study within artificial intelligence that focuses on the development and study of statistical algorithms capable of learning from data and making predictions or decisions without being explicitly programmed to perform specific tasks. It involves the use of algorithms and statistical models to analyze and draw inferences from patterns in data. Machine learning is widely used in various applications, including image and speech recognition, recommendation systems, and autonomous vehicles, among others.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study within artificial intelligence that focuses on the development and study of statistical algorithms capable of learning from data and making predictions or decisions without being explicitly programmed to perform specific tasks. It involves the use of algorithms and statistical models to analyze and draw inferences from patterns in data. Machine learning is widely used in various applications, including image and speech recognition, recommendation systems, and autonomous vehicles, among others.'}

In [26]:

agent_executer.invoke({"input":"What's the paper 1605.08386 about?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, published on May 26, 2016, studies graphs on lattice points where the edges are derived from a finite set.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, published on May 26, 2016, studies graphs on lattice points where the edges are derived from a finite set.'}